## Background

A team of researchers recently [published a paper](https://ml-site.cdn-apple.com/papers/the-illusion-of-thinking.pdf) claiming that large reasoning models (LRMs) are not capable of "thinking" or making logical choices using some simple logical puzzles. In the paper, LLMs (Large Language Models) and LRMs are both capable of solving the puzzles when low-complexity versions of the puzzles are presented, but when high-complexity versions are presented, both LRMs and LLMs are incapable of logic. 

In this post, I'll be referring to the Tower of Hanoi puzzle, one of the puzzles used in the paper. The rules and an explanation of the game can be found on [Wikipedia](https://en.wikipedia.org/wiki/Tower_of_Hanoi). 

In this post, I'll attempt to modify the experiment design to account for two possible flaws in the paper's methodology: 
1. Perhaps the language models were failing simply because the context is too small
2. Perhaps the language models were failing because they did not have a way to update and understand the "current state" of the problem. 

I had some personal experience with the second potential flaw when, in an attempt to prove that I am smarter than a machine, I tried to write out all of the moves necessary to solve the Tower of Hanoi with 9 rings. I ended up spending a lot of time trying to go back through moves and remember what the "current state" was. However, when I solved the puzzle in a way that I could always see the "current state", I was a lot faster. 




## Game setup

Okay, here we will set up some functions so that the LLM can play our game. 

In [60]:
import numpy as np
import re
STACK_OFFSET = 3 ## An offset from the "current state" string to identify where the stacks start
STACK_LETTERS = "ABC" ## All of the letters in stacks
INCORRECT_MOVE_LANGUAGE = "Invalid move. "

def process_move(move, previous_state):
    """
    Take a move an either return the state after the move was created or return None.
    """
    if len(move) != 2:
        return None
    move_disc = move[0]
    move_stack = move[1]
    
    
    if not move_disc.isdigit():
        return None
    if move_stack not in STACK_LETTERS:
        return None
    previous_state_list = previous_state.split("\n")
    final_discs = [previous_state_list[STACK_OFFSET + i].strip()[-1] for i in range(3)]
    if move_disc not in final_discs:

        return None
    final_disc_stack = previous_state_list[STACK_OFFSET + STACK_LETTERS.index(move_stack)]
    
    if final_discs[STACK_LETTERS.index(move_stack)].isdigit() and (move_disc > final_discs[STACK_LETTERS.index(move_stack)]):
        return None
    
    previous_state_list[STACK_OFFSET + final_discs.index(move_disc)] = previous_state_list[STACK_OFFSET + final_discs.index(move_disc)][:-2]
    previous_state_list[STACK_OFFSET + STACK_LETTERS.index(move_stack)] += " " + move_disc
    
    return "\n".join(previous_state_list)
    
    
    

def get_next_state(previous_state, response):
    """
    Get the next state from a move, calling process_move. Parse the move from a LLM response, and 
    """
    
    move = re.sub("[^\w\s]", "", response).strip().split(" ")[0]
    next_state = process_move(move, previous_state)
    if next_state is not None:
        return next_state.replace(INCORRECT_MOVE_LANGUAGE, "")
    else:
        return INCORRECT_MOVE_LANGUAGE + previous_state.replace(INCORRECT_MOVE_LANGUAGE, "")
    
    
def get_valid_moves(current_state):
    """
    Get a list of possible valid moves based on the current staate of the game
    """
    end_chars = [x.strip()[-1] for x in current_state.split("\n")[STACK_OFFSET:STACK_OFFSET+3]]
    valid_moves = []
    for i, disc in enumerate(end_chars):
        if disc.isdigit():
            for j in range(3):
                if j == i:
                    continue
                if end_chars[j] <= disc:
                    continue
                valid_moves.append(disc + STACK_LETTERS[j])
    return valid_moves

def get_random_move(current_state):
    """Choose a random move"""
    return np.random.choice(get_valid_moves(current_state))

def get_full_stack(num_discs):
    """Get a full stack of discs together in the proper order. To be used in building a system prompt."""
    full_stack = ""
    for i in range(num_discs):
        full_stack = " " + str(i) + full_stack
    return full_stack


def get_game_start_end(num_discs):
    """
    Get starting and final states of the game. 
    """
    full_stack = get_full_stack(num_discs)
    starting_game = f"""The current state of the game is:

```
A | {full_stack}
B | 
C | 
```"""
    
    ending_game = f"""The current state of the game is:

```
A | 
B | 
C | {full_stack}
```"""
    return starting_game, ending_game
    
                
    
    
    

I used this chunk to debug the gameplay and demonstrate how long it takes for a model to solve the puzzle at random.

In [61]:
current_state, ending_state = get_game_start_end(2)
num_iterations = 0
num_discs = 3

current_state, ending_state = get_game_start_end(num_discs)

## Looking at 
while current_state != ending_state:
    num_iterations += 1
    if num_iterations > 20:
        break
    move = get_random_move(current_state)
    current_state = get_next_state(current_state, move)
    
    print(current_state)
    print(get_valid_moves(current_state))
    
print(f"Completed the game at {num_iterations} iterations")

The current state of the game is:

```
A |  2 1
B |  0
C | 
```
['1C', '0A', '0C']
The current state of the game is:

```
A |  2 1
B | 
C |  0
```
['1B', '0A', '0B']
The current state of the game is:

```
A |  2
B |  1
C |  0
```
['1A', '0A', '0B']
The current state of the game is:

```
A |  2 1
B | 
C |  0
```
['1B', '0A', '0B']
The current state of the game is:

```
A |  2 1 0
B | 
C | 
```
['0B', '0C']
The current state of the game is:

```
A |  2 1
B | 
C |  0
```
['1B', '0A', '0B']
The current state of the game is:

```
A |  2 1 0
B | 
C | 
```
['0B', '0C']
The current state of the game is:

```
A |  2 1
B | 
C |  0
```
['1B', '0A', '0B']
The current state of the game is:

```
A |  2 1
B |  0
C | 
```
['1C', '0A', '0C']
The current state of the game is:

```
A |  2
B |  0
C |  1
```
['0A', '0C', '1A']
The current state of the game is:

```
A |  2 1
B |  0
C | 
```
['1C', '0A', '0C']
The current state of the game is:

```
A |  2 1 0
B | 
C | 
```
['0B', '0C']
The current state of t

Now, we're ready to set up a function that calls the LLM to play the game. 

In [42]:
from config import OPENAI_API_KEY
import os
from openai import OpenAI
import re
import time

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def llm_plays_tower_hanoi(num_discs, model = "gpt-4.1", restrict_token_length = False, max_num_iterations = 50, seconds_sleep_per_step = 1, temperature = 1):
    """Have an openAI model play the tower of hanoi
    
    Args:
        num_discs: The number of discs to include in the game
        model: The model to use
        restrict_token_length: Whether to restrict the model's input to only include the most recent game state. If True, the model will only receive the current state
        max_num_iterations: the maximum number of iterations. 
        seconds_sleep_per_step: The number of seconds to sleep for each move in gameplay. Useful if you are hitting rate limits. 
        temperature: The model's temperature
    """
    sys_prompt = f"""
You are a puzzle master completing the tower of Hanoi puzzle. 

This puzzle will be represented to you in this fashion:

A | {get_full_stack(num_discs)}
B | 
C | 

There are three stacks: A, B, and C. 

There are {num_discs} discs on the stacks, represented as numbers, with 0 being the smallest and {num_discs - 1} being the largest. Each digit is a disc

Your goal is to move all of the discs from stack A to stack C, in the proper order. 

You must follow the following rules: 
1. Only one disk may be moved at a time.
2. Each move consists of taking the top disk from one of the stacks and placing it on the top of another stack or on an empty stack.
3. No disk may be placed on top of a disk that is smaller than it. (so, disk 1 cannot be placed on top of disk 0)

The game will be represented to you in an intermediate state, where the right-most disc represents the "top" of each stack. 


At the start of the game, the puzzle looked like:

A | {get_full_stack(num_discs)}
B | 
C | 

To win, you must move all the discs to stack C, so the puzzle will look like:

A | 
B | 
C | {get_full_stack(num_discs)}

Your role is to plan ahead and make the best move based on the current state of the puzzle, so that the puzzle moves closer to the winning state. 

To make a move, state the number of the disk and the stack you want to move it to in the following manner:
```
1B
```

The move `1B` moves the disc 1 to the top of stack B. The move `0C` moves the disc 0 to the top of stack C.

Do not state anything other than the move in your response.
"""

    print(sys_prompt)


    client = OpenAI()



    current_state, ending_state = get_game_start_end(num_discs)
    num_iterations = 0

    messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": current_state}
            ]


    while current_state != ending_state:
        valid_moves = "\nValid Moves are: " + "[`" + "`, `".join(get_valid_moves(current_state)) + "`]"
        
        num_iterations += 1
        if num_iterations >=  max_num_iterations:
            break


        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=messages
        )
        move = response.choices[0].message.content
        print("-"*20)
        print(current_state + valid_moves)
        print("-"*20)
        print(move)

        
        messages.append({"role": "assistant", "content": move})
        
        previous_state = current_state + valid_moves
        previous_move = move
        current_state = get_next_state(current_state, move)
        
        if restrict_token_length:
            ## Only pass in the current state of the game
            messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": previous_state.replace(INCORRECT_MOVE_LANGUAGE, "")},
            {"role": "user", "content": previous_move},
            {"role": "user", "content": current_state + valid_moves}
            ]
        else:
            # Append the new game state as a user message
            messages.append({"role": "user", "content": current_state + valid_moves})
            
        time.sleep(seconds_sleep_per_step) ## Prevent myself from hitting rate limits.
    if current_state == ending_state:
        
        print(f"Completed the game at {num_iterations} iterations")
    else:
        print(f"Stoped gameplay by reaching the max number of iterations ({max_num_iterations})")

One note about tokenization: I put spaces between the "disc" numbers in the stack to ensure that those numbers are broken up into different tokens, but the actual moves (e.g. "3C") don't need a space between them because the letters and numbers are broken into different tokens automatically. 

If you're curious aabout how text gets tokenized for openAI models, you can check out this [cool tool](https://platform.openai.com/tokenizer) that visualizes how things get tokenized.

Experiment 1: the model plays the game with context of all previous moves. 

In [35]:
llm_plays_tower_hanoi(num_discs = 3)




You are a puzzle master completing the tower of Hanoi puzzle. 

This puzzle will be represented to you in this fashion:

A |  2 1 0
B | 
C | 

There are three stacks: A, B, and C. 

There are 3 discs on the stacks, represented as numbers, with 0 being the smallest and 2 being the largest. Each digit is a disc

Your goal is to move all of the discs from stack A to stack C, in the proper order. 

You must follow the following rules: 
1. Only one disk may be moved at a time.
2. Each move consists of taking the top disk from one of the stacks and placing it on the top of another stack or on an empty stack.
3. No disk may be placed on top of a disk that is smaller than it. (so, disk 1 cannot be placed on top of disk 0)

The game will be represented to you in an intermediate state, where the right-most disc represents the "top" of each stack. 


At the start of the game, the puzzle looked like:

A |  2 1 0
B | 
C | 

To win, you must move all the discs to stack C, so the puzzle will look li

Experiment 2: does lowering the temperature enable the model to better make moves?

In [40]:
llm_plays_tower_hanoi(num_discs = 3, temperature = 0.1)


You are a puzzle master completing the tower of Hanoi puzzle. 

This puzzle will be represented to you in this fashion:

A |  2 1 0
B | 
C | 

There are three stacks: A, B, and C. 

There are 3 discs on the stacks, represented as numbers, with 0 being the smallest and 2 being the largest. Each digit is a disc

Your goal is to move all of the discs from stack A to stack C, in the proper order. 

You must follow the following rules: 
1. Only one disk may be moved at a time.
2. Each move consists of taking the top disk from one of the stacks and placing it on the top of another stack or on an empty stack.
3. No disk may be placed on top of a disk that is smaller than it. (so, disk 1 cannot be placed on top of disk 0)

The game will be represented to you in an intermediate state, where the right-most disc represents the "top" of each stack. 


At the start of the game, the puzzle looked like:

A |  2 1 0
B | 
C | 

To win, you must move all the discs to stack C, so the puzzle will look li

--------------------
Invalid move. The current state of the game is:

```
A | 
B |  1 0
C |  2
```
Valid Moves are: [`0A`, `0C`, `2A`]
--------------------
0A
--------------------
The current state of the game is:

```
A |  0
B |  1
C |  2
```
Valid Moves are: [`0B`, `0C`, `1C`]
--------------------
0C
--------------------
The current state of the game is:

```
A | 
B |  1
C |  2 0
```
Valid Moves are: [`1A`, `0A`, `0B`]
--------------------
1C
--------------------
Invalid move. The current state of the game is:

```
A | 
B |  1
C |  2 0
```
Valid Moves are: [`1A`, `0A`, `0B`]
--------------------
0B
Stoped gameplay by reaching the max number of iterations (50)


Experiment 3: can the game run if we remove previous context?

In [43]:
llm_plays_tower_hanoi(num_discs = 3, restrict_token_length  = True)


You are a puzzle master completing the tower of Hanoi puzzle. 

This puzzle will be represented to you in this fashion:

A |  2 1 0
B | 
C | 

There are three stacks: A, B, and C. 

There are 3 discs on the stacks, represented as numbers, with 0 being the smallest and 2 being the largest. Each digit is a disc

Your goal is to move all of the discs from stack A to stack C, in the proper order. 

You must follow the following rules: 
1. Only one disk may be moved at a time.
2. Each move consists of taking the top disk from one of the stacks and placing it on the top of another stack or on an empty stack.
3. No disk may be placed on top of a disk that is smaller than it. (so, disk 1 cannot be placed on top of disk 0)

The game will be represented to you in an intermediate state, where the right-most disc represents the "top" of each stack. 


At the start of the game, the puzzle looked like:

A |  2 1 0
B | 
C | 

To win, you must move all the discs to stack C, so the puzzle will look li

--------------------
The current state of the game is:

```
A |  0
B |  2
C |  1
```
Valid Moves are: [`0B`, `0C`, `1B`]
--------------------
0B
--------------------
The current state of the game is:

```
A | 
B |  2 0
C |  1
```
Valid Moves are: [`0A`, `0C`, `1A`]
--------------------
1B
--------------------
Invalid move. The current state of the game is:

```
A | 
B |  2 0
C |  1
```
Valid Moves are: [`0A`, `0C`, `1A`]
--------------------
0C
Stoped gameplay by reaching the max number of iterations (50)


I'm a bit surprised at how bad the LLM is at playing the tower of hanoi, so here I'll replicate Apple's paper to see if that may be different. 

In [57]:
def replicate_apple_procedure(num_discs, model = "gpt-4.1"):

    apple_replication_sys_prompt = f"""You are a helpful assistant. Solve this puzzle for me.
    There are three pegs and {num_discs} disks of different sizes stacked on the first peg. The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
    1. Only one disk can be moved at a time.
    2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
    3. A larger disk may not be placed on top of a smaller disk.
    The goal is to move the entire stack to the third peg.
    Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
    moves = [[1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2]]

    Thismeans: Move disk  1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on.
    Requirements:
    • When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
    • The positions are 0-indexed (the leftmost peg is 0).
    • Ensure your final answer includes the complete list of moves in the format:
           moves = [[disk id, from peg, to peg], ...]
    """

    apple_replication_user_prompt = f"""
    The initial state of the game is: [[{",".join([str(num_discs - i ) for i in range(num_discs)])}], [], []]

    The winning state of the game is: [[], [], [{",".join([str(num_discs - i ) for i in range(num_discs)])}]]

    Share your complete list of moves to move from the initial state to the winning state
    """


    messages = [
                {"role": "system", "content": apple_replication_sys_prompt},
        {"role": "user", "content": apple_replication_user_prompt}
    ]


    client = OpenAI()

    response = client.chat.completions.create(
                model=model,
                messages=messages
            )

    print(apple_replication_sys_prompt)
    print("-"*20)
    print(apple_replication_user_prompt)
    print("-"*20)
    print(response.choices[0].message.content)



In [58]:
replicate_apple_procedure(num_discs = 3)

You are a helpful assistant. Solve this puzzle for me.
    There are three pegs and 3 disks of different sizes stacked on the first peg. The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
    1. Only one disk can be moved at a time.
    2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
    3. A larger disk may not be placed on top of a smaller disk.
    The goal is to move the entire stack to the third peg.
    Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
    moves = [[1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2]]

    Thismeans: Move disk  1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on.
    Requirements:
    • When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
    • The positions are

And here, I'll do one final demonstration to show that I am in fact smarter than a machine. I bet the model cannot solve the 9-disc puzzle that I did

In [59]:
replicate_apple_procedure(num_discs = 9)

You are a helpful assistant. Solve this puzzle for me.
    There are three pegs and 9 disks of different sizes stacked on the first peg. The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
    1. Only one disk can be moved at a time.
    2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
    3. A larger disk may not be placed on top of a smaller disk.
    The goal is to move the entire stack to the third peg.
    Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
    moves = [[1, 0, 2], [2, 0, 1], [1, 2, 1], [3, 0, 2], [1, 1, 0], [2, 1, 2], [1, 0, 2]]

    Thismeans: Move disk  1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on.
    Requirements:
    • When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
    • The positions are

## Conclusions:

This was a fun way to replicate and extend the paper on language model reasoning. 
1. This replication reduces my confidence in LLM's ability to reason through problems more than the original paper did. If LLMs are not able to reliably choose the next step to solve the Tower of Hanoi when the current state is presented to them, I am not confident that there is any logic/reasoning happening. 
2. Models perform better at puzzles with many instances of the previous state in their context than with just 2 previous state instances. 
3. LLMs seem to be better at giving solutions to models all at once, rather than looking at the current state of the problem and making incremental fixes. This does seem to be a fundamental weakness in LLMs, as a lot of real-world work that happens takes place in incremental fashions, where you are not exactly at the start state or end state of a problem.


## Future steps:

1. If I had more money to burn on openAI credits, I would have tried this with one of OpenAI's reasoning models instead of gpt-4.1. If any of you are curious and want to extend this work with a a reasoning model, I would be interested to hear about it!
2. It might be interesting to extend the length of the context for the models that are provided with the limited context window and increase the number of discs in the tower. I did not do this because performance was already bad with 3 discs, but if you try this, let me know about it.
3. The system prompt could probably be improved to improve model responsiveness. 

